In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('weight-height-knn.csv')

In [3]:
df.shape

(8555, 3)

In [4]:
df.isnull().sum()

Gender    0
Height    0
Weight    0
dtype: int64

In [5]:
df.head()

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df.drop('Weight',axis=1)

In [8]:
y = df['Weight']

In [9]:
import category_encoders as ce

In [10]:
binary_encoder = ce.BinaryEncoder(cols=['Gender'])
X = binary_encoder.fit_transform(X)

X.head()

C:\Users\sumon\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Gender_0,Gender_1,Height
0,0,1,73.847017
1,0,1,68.781904
2,0,1,74.110105
3,0,1,71.730978
4,0,1,69.881796


In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X['Height'] = scaler.fit_transform(X[['Height']])

X.head()

,Gender_0,Gender_1,Height
0,0,1,1.827233
1,0,1,0.512039
2,0,1,1.895545
3,0,1,1.277788
4,0,1,0.797634


In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .30,random_state = 42)

In [13]:
X_train.shape

(5988, 3)

In [14]:
y_train.shape

(5988,)

In [15]:
X_train.head()

,Gender_0,Gender_1,Height
553,0,1,0.203599
1397,0,1,1.244219
7934,1,0,-1.086391
8367,1,0,-0.174956
3320,0,1,1.306955


In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [18]:
from sklearn.metrics import mean_squared_error,r2_score

In [19]:
prediction = reg.predict(X_test)

In [20]:
mse_linear = mean_squared_error(y_test,prediction)

In [21]:
r2_score_linear = r2_score(y_test, prediction)

In [22]:
from sklearn.neighbors import KNeighborsRegressor

In [23]:
neigh = KNeighborsRegressor()

In [24]:
neigh.fit(X_train,y_train)

KNeighborsRegressor()

In [25]:
prediction_neigh = reg.predict(X_test)

In [26]:
mse_neigh = mean_squared_error(y_test,prediction_neigh)
r2_score_neigh = r2_score(y_test, prediction_neigh)


In [27]:
print('MSE (Linear) = %f r2 (Linear)=%f' %(mse_linear,r2_score_linear))
print('MSE (KNN) = %f r2 (KNN)=%f' %(mse_neigh,r2_score_neigh))

MSE (Linear) = 96.837344 r2 (Linear)=0.905911
MSE (KNN) = 96.837344 r2 (KNN)=0.905911


In [28]:
n_neighbors = [int(i) for i in np.linspace(1,50,30)]

[1,
 2,
 4,
 6,
 7,
 9,
 11,
 12,
 14,
 16,
 17,
 19,
 21,
 22,
 24,
 26,
 28,
 29,
 31,
 33,
 34,
 36,
 38,
 39,
 41,
 43,
 44,
 46,
 48,
 50]

In [29]:
grids = {
 'n_neighbors' : n_neighbors   
}

In [30]:
from sklearn.model_selection import RandomizedSearchCV

In [31]:
neigh_opt = KNeighborsRegressor()
rmcv = RandomizedSearchCV(neigh_opt,grids,n_iter=200,cv=3)

In [32]:
rmcv.fit(X_train,y_train)

C:\Users\sumon\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 30 is smaller than n_iter=200. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=KNeighborsRegressor(), n_iter=200,
                   param_distributions={'n_neighbors': [1, 2, 4, 6, 7, 9, 11,
                                                        12, 14, 16, 17, 19, 21,
                                                        22, 24, 26, 28, 29, 31,
                                                        33, 34, 36, 38, 39, 41,
                                                        43, 44, 46, 48, 50]})

In [33]:
rmcv.best_params_

{'n_neighbors': 46}

In [34]:
neigh_opt = rmcv.best_estimator_

In [35]:
neigh_opt.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=46)

In [36]:
prediction_neigh_opt = neigh_opt.predict(X_test)
mse_neigh_opt = mean_squared_error(y_test,prediction_neigh_opt)
r2_score_neigh_opt = r2_score(y_test, prediction_neigh_opt)

In [37]:
print('MSE (Linear) = %f r2 (Linear)=%f' %(mse_linear,r2_score_linear))
print('MSE (KNN) = %f r2 (KNN)=%f' %(mse_neigh,r2_score_neigh))
print('MSE (KNN Opt) = %f r2 (KNN Opt)=%f' %(mse_neigh_opt,r2_score_neigh_opt))

MSE (Linear) = 96.837344 r2 (Linear)=0.905911
MSE (KNN) = 96.837344 r2 (KNN)=0.905911
MSE (KNN Opt) = 99.263700 r2 (KNN Opt)=0.903554
